In [115]:
import json

# 从 PandaLM 的训练数据 train.json 中抽样的 35000 条
with open("./sampled_train.json", "r") as f:
  pandaLM_sample_data = json.load(f)

autoJ_pairwise = []
autoJ_single = []
with open("./training/pairwise_traindata.jsonl", "r") as f:
  for line in f:
    autoJ_pairwise.append(json.loads(line))

with open("./training/single_traindata.jsonl", "r") as f:
  for line in f:
    autoJ_single.append(json.loads(line))

data = []

In [116]:
for i in range(len(pandaLM_sample_data)):
  input_sequence = pandaLM_sample_data[i]['input_sequence']
  output_sequence = pandaLM_sample_data[i]['output_sequence']

  data.append({
    "instruction": input_sequence,
    "output": output_sequence
  })

In [117]:
data[0]

{'instruction': 'Below are two responses for a given task. The task is defined by the Instruction. Evaluate the responses and generate a reference answer for the task.\n\n### Instruction:\nGenerate a new expression to describe a person who is always late.\n\n### Response 1:\nNever on time.\n\n### Response 2:\nHe is always late.\n\n### Evaluation:\n',
 'output': '1\n\n### Reason: Response 1 is a more creative and concise way to describe a person who is always late.\n\n### Reference: Chronic latecomer.\n'}

In [118]:
autoJ_pairwise[0]

{'usrmsg': 'You are assessing two submitted responses on a given user\'s query and judging which response is better or they are tied. Here is the data:\n\n[BEGIN DATA]\n***\n[Query]: my powershell script keeps stopping until i press enter, how can I make sure it runs all commands without waiting? I need to run this automatically without user input\n***\n[Response 1]: To prevent your PowerShell script from stopping until you press enter, you can use the `-noexit` parameter to suppress the exit prompt.\n\nHere\'s an example of how you can modify your script to use the `-noexit` parameter:\n\n```PowerShell\n$noexit = $true\n\nwhile ($noexit) {\n    # Your script code here\n}\n```\n\nWhen you run this modified script, it will continue to execute without stopping until you press enter, as opposed to having the script exit immediately after the first command.\n***\n[Response 2]: You can try adding the "-NoInteractive" flag to your PowerShell command. This flag will prevent any prompts or int

In [119]:
autoJ_single[0]

{'usrmsg': 'Write critiques for a submitted response on a given user\'s query, and grade the response:\n  \n[BEGIN DATA]\n***\n[Query]: Evaluate the efficiency of the popular programming languages you have got to know.\n***\n[Response]: As an AI language model, I do not have personal opinions, but I can provide information on popular programming languages and their efficiency.\n\nHere is a list of commonly used programming languages, along with their Ease of Use (EOI) scores, which is a measure of how easy a programming language is to learn, perform, and use:\n\n* Python: 90\n* JavaScript: 83\n* C++: 88\n* PHP: 78\n* Perl: 78\n* PHP and MySQL: 77\n* Ruby: 67\n* Swift: 67\n* Java: 67\n* Rust: 67\n* C#: 67\n* Go: 67\n* Swift: 66\n* Python: 66\n* PHP: 54\n* Perl: 54\n* Python with Pandas: 54\n* JavaScript with React: 54\n* C++ with OpenMP: 54\n* Rust with Ruststd: 54\n* Go with Golang: 54\n* Swift with SwiftUI: 54\n* C# with.NET Core: 54\n* Rust with go2e: 54\n* Go with rust: 54\n* C#: 5.

In [120]:
print(autoJ_single[0]['usrmsg'])

Write critiques for a submitted response on a given user's query, and grade the response:
  
[BEGIN DATA]
***
[Query]: Evaluate the efficiency of the popular programming languages you have got to know.
***
[Response]: As an AI language model, I do not have personal opinions, but I can provide information on popular programming languages and their efficiency.

Here is a list of commonly used programming languages, along with their Ease of Use (EOI) scores, which is a measure of how easy a programming language is to learn, perform, and use:

* Python: 90
* JavaScript: 83
* C++: 88
* PHP: 78
* Perl: 78
* PHP and MySQL: 77
* Ruby: 67
* Swift: 67
* Java: 67
* Rust: 67
* C#: 67
* Go: 67
* Swift: 66
* Python: 66
* PHP: 54
* Perl: 54
* Python with Pandas: 54
* JavaScript with React: 54
* C++ with OpenMP: 54
* Rust with Ruststd: 54
* Go with Golang: 54
* Swift with SwiftUI: 54
* C# with.NET Core: 54
* Rust with go2e: 54
* Go with rust: 54
* C#: 5.4
* Java with JavaDoc: 4.2
* JavaScript with Rea

In [121]:
print(autoJ_pairwise[0]['usrmsg'])

You are assessing two submitted responses on a given user's query and judging which response is better or they are tied. Here is the data:

[BEGIN DATA]
***
[Query]: my powershell script keeps stopping until i press enter, how can I make sure it runs all commands without waiting? I need to run this automatically without user input
***
[Response 1]: To prevent your PowerShell script from stopping until you press enter, you can use the `-noexit` parameter to suppress the exit prompt.

Here's an example of how you can modify your script to use the `-noexit` parameter:

```PowerShell
$noexit = $true

while ($noexit) {
    # Your script code here
}
```

When you run this modified script, it will continue to execute without stopping until you press enter, as opposed to having the script exit immediately after the first command.
***
[Response 2]: You can try adding the "-NoInteractive" flag to your PowerShell command. This flag will prevent any prompts or interactive input during the executio

In [122]:
single = """Write critiques for a submitted response on a given user's query, and grade the response:

[BEGIN DATA]
***
[Query]: {prompt}
***
[Response]: {response}
***
[END DATA]

Write critiques for this response. After that, you should give a final rating for the response on a scale of 1 to 10 by strictly following this format: "[[rating]]", for example: "Rating: [[5]]"."""

pairwise_tie = """You are assessing two submitted responses on a given user's query and judging which response is better or they are tied. Here is the data:

[BEGIN DATA]
***
[Query]: {prompt}
***
[Response 1]: {response}
***
[Response 2]: {response_another}
***
[END DATA]

Here are the instructions to assess and compare the two responses:

1. Pinpoint the key factors to distinguish these two responses.
2. Conclude your comparison by providing a final decision on which response is better, or they are tied. Begin your final decision statement with "So, the final decision is Response 1 / Response 2 / Tie". Ensure that your decision aligns coherently with the comprehensive evaluation and comparison you've provided."""

In [123]:
for i in range(len(autoJ_pairwise)):
  usrmsg = autoJ_pairwise[i]['usrmsg']
  target_output = autoJ_pairwise[i]['target_output']

  data.append({
    "instruction": usrmsg,
    "output": target_output
  })


for i in range(len(autoJ_single)):
  usrmsg = autoJ_single[i]['usrmsg']
  target_output = autoJ_single[i]['target_output']

  data.append({
    "instruction": usrmsg,
    "output": target_output
  })

In [124]:
len(data)

39396

In [125]:
import random

random.seed(42)

In [126]:
random.shuffle(data)
data[0]

{'instruction': 'Below are two responses for a given task. The task is defined by the Instruction with an Input that provides further context. Evaluate the responses and generate a reference answer for the task.\n\n### Instruction:\nReturn the vowel count for the following word: "Apple".\n\n### Input:\nApple\n\n### Response 1:\n1\n\n### Response 2:\n2\n\n### Evaluation:\n',
 'output': '2\n\n### Reason: Response 2 is correct because the word "Apple" has two vowels, \'a\' and \'e\'.\n\n### Reference: 1\n'}

In [132]:
with open("final_train_data.json", "w") as f:
  json.dump(data, f, indent=4)

In [136]:
max = 0
sum = 0

for i in range(len(data)):
  t = len(data[i]['instruction']) + len(data[i]['output'])
  sum += t
  if max <= t:
    max = t

mean = sum / len(data)

print(max)
print(mean)

11991
1397.1250126916439
